<a href="https://colab.research.google.com/github/bluevit/NER-Testing/blob/main/NER_for_ResumeAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Training spaCy NER for Resume Analysis</h3>


In [1]:
!pip install numpy==1.25

In [ ]:
!pip install -U spacy==3.7.2
!pip install spacy-transformers

In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [4]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6 (from 1)
Receiving objects: 100% (82/82), 5.62 MiB | 20.99 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [5]:
import json

cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json','r'))

In [6]:
len(cv_data)

200

In [ ]:
# you can see the data(json) by uncommenting below statement
#cv_data

In [7]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


---
config.cfg must be modified in order to reduce the training time<br>
(generated in *content/CV-Parsing-using-Spacy-3/data/training/config.cfg*)<br><br>
max_steps parameter must be set to some low number like 2000 or 4000, which will drastically reduce the training time.</h3>
<hr>

This function takes a dataset of text and annotations, processes each text to identify and label entities, and stores the results in a DocBin object that can be used for further training or analysis with spaCy.

In [8]:
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue
      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + ' ' + str(text) + '\n'
        file.write(err_data)
      else :
        ents.append(span)
    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass
      #print(text)
  return db

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [10]:
file = open('error.txt', 'w')
db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
100%|██████████| 60/60 [00:00<00:00, 67.79it/s]


In [11]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:

<h3>Testing the Trained Model</h3>

In [12]:
nlp = spacy.load('/content/output/model-best')

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We

In [14]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 85.2 MB/s eta 0:00:00


In [15]:
import sys , fitz

In [16]:
fname = '/content/CV-Parsing-using-Spacy-3/data/test/Alice Clark CV.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
  text = text + str(page.get_text())
doc.close()

In [17]:
text = text.strip()

In [18]:
text

'Alice Clark \nAI / Machine Learning \n \nDelhi, India Email me on Indeed \n• \n20+ years of experience in data handling, design, and development \n• \nData Warehouse: Data analysis, star/snow flake scema data modelling and design specific to \ndata warehousing and business intelligence \n• \nDatabase: Experience in database designing, scalability, back-up and recovery, writing and \noptimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. \nCloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, \nStream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake \nanalytics(U-SQL) \nWilling to relocate anywhere \n \nWORK EXPERIENCE \nSoftware Engineer \nMicrosoft – Bangalore, Karnataka \nJanuary 2000 to Present \n1. Microsoft Rewards Live dashboards: \nDescription: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping \nonline. Microsoft Rewards members can earn points when 

In [19]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text, '---->' , ent.label_)

Alice Clark ----> Name
Delhi ----> Location
Microsoft ----> Companies worked at
Software Engineer ----> Designation
Microsoft ----> Companies worked at
Microsoft ----> Companies worked at
Microsoft ----> Companies worked at
Microsoft ----> Companies worked at
Microsoft ----> Companies worked at
Indian Institute of Technology ----> College Name
Machine Learning, Natural Language Processing, and Big Data Handling ----> Skills


In [20]:
!zip -r /content/output.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/model-best/ (stored 0%)
  adding: content/output/model-best/config.cfg (deflated 61%)
  adding: content/output/model-best/ner/ (stored 0%)
  adding: content/output/model-best/ner/model (deflated 8%)
  adding: content/output/model-best/ner/cfg (deflated 33%)
  adding: content/output/model-best/ner/moves (deflated 73%)
  adding: content/output/model-best/meta.json (deflated 66%)
  adding: content/output/model-best/transformer/ (stored 0%)
  adding: content/output/model-best/transformer/model (deflated 13%)
  adding: content/output/model-best/transformer/cfg (stored 0%)
  adding: content/output/model-best/tokenizer (deflated 81%)
  adding: content/output/model-best/vocab/ (stored 0%)
  adding: content/output/model-best/vocab/strings.json (deflated 76%)
  adding: content/output/model-best/vocab/key2row (stored 0%)
  adding: content/output/model-best/vocab/vectors.cfg (stored 0%)
  adding: content/output/model-best/vocab/vectors

---
<h4>For Reference:</h4>
<a href='https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git'>CV-Parsing-using-Spacy-3
</a><br>
<a href='https://github.com/yashlikescode/spacyResumeParcer.git'>spacyResumeParcer</a>
<hr>